In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import silhouette_samples
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import ward, dendrogram, complete, average, single

import geopandas as gpd
import pygeos
import gpdvega 

import altair as alt


In [ ]:
#Load on California shape data
counties = gpd.read_file('/work/ca-county-boundaries/CA_Counties/CA_Counties_TIGER2016.shp')
counties['county_name'] = counties['NAME'].str.lower()
counties.head()

In [ ]:
#Load in California data
ca_df = pd.read_csv('/work/cleaned-csvs/ca_counties_full_dataset.csv')

ca_df.columns

In [ ]:
# Check for sparse columns
nullseries=ca_df.isna().sum()
nullseries[nullseries > 0]

In [ ]:
#Data prep for unsupervised learning tasks

#Let's just look at one year of data, 2018
ca_df = ca_df[ca_df['year']==2018]

#Before removing text columns, create a mapper for county names
ca_df = ca_df.reset_index()
county_list_mapper = list(ca_df['county_name'])
county_dict_mapper = (ca_df[['county_name']].to_dict('index'))

#Identify and drop non-numeric fields
non_numeric = ca_df.select_dtypes(exclude='number').columns.to_list()
ca_df = ca_df.drop(columns=non_numeric)


#Remove fields that are numeric, but not meaningful features of the county
ca_df = ca_df.drop(columns = {'state','county', 'longitude',
       'latitude','year'})


ca_df.shape

In [ ]:
ca_df.columns

### Feature Scaling
Let's try using the standard scaler, which follows the Standard Normal Distribution an makes the mean of each field 0.  

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(ca_df)

# verify that there are no missing values and inspect data
print('missing values:', pd.DataFrame(scaled_data).isna().sum().sum())
pd.DataFrame(scaled_data)


In [ ]:
scaled_data.shape

### K-Means Clustering

We will run K-Means with multiple n_clusters values so we can use the elbow method. To determine the optimal number of clusters, we have to select the value of k at the “elbow”, or the point after which the inertia start decreasing in a linear fashion. 

Inertia is the sum of squared distances of samples to their closest cluster center (SSE)

We will try two methods of initializing K-Means, the default, which is 'k-means++', which selectes inital cluster centers in a "smart way", and 'random' which choses initial centroids at random.

We will also look at two metrics to check the quality of our clusters:

#### Davies_Bouldin Index
This index signifies the average ‘similarity’ between clusters, where the similarity is a measure that compares the distance between clusters with the size of the clusters themselves. Zero is the lowest possible score. Values closer to zero indicate a better partition.

#### Calinski-Harabasz Index
The index is the ratio of the sum of between-clusters dispersion and of within-cluster dispersion for all clusters (where dispersion is defined as the sum of distances squared). The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.



In [ ]:
k_range = [*range(3, 20)]

#Calculate inertia scores using the default initialization 'k-means++'
inertia_scores = []
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42,init ='k-means++').fit(scaled_data)
    inertia_scores.append(kmeans.inertia_)
    kmeans.fit(scaled_data)
    ch_score = metrics.calinski_harabasz_score(scaled_data,kmeans.labels_)
    db_score = metrics.davies_bouldin_score(scaled_data,kmeans.labels_)
    print('k: {}, ch_score: {}, db_score {} '.format(k,ch_score,db_score))


Our Calinksi-Harabasz scores are all low.

inertia_ provides the sum of the squared error, SSE. We will plot this to find "elbow"


In [ ]:
plt.plot(k_range, inertia_scores, '-o')
plt.ylabel('SSE')
plt.xlabel('Number of clusters (k)')
plt.title('The Elbow Method using "k-means++" initialization ')

Not seeing much of an elbow above, let's try using the random initialization

In [ ]:
inertia_scores = []
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42,init='random').fit(scaled_data)
    inertia_scores.append(kmeans.inertia_)
    kmeans.fit(scaled_data)
    ch_score = metrics.calinski_harabasz_score(scaled_data,kmeans.labels_)
    db_score = metrics.davies_bouldin_score(scaled_data,kmeans.labels_)
    print('k: {}, ch_score: {}, db_score {} '.format(k,ch_score,db_score))



Our Calinski-Harabasz scores are still low. Let's see if the elbow shows us anything different...

In [ ]:
plt.plot(k_range, inertia_scores, '-o')
plt.ylabel('SSE')
plt.xlabel('Number of clusters (k)')
plt.title('The Elbow Method using "random" initialization')

Neither of these really show an elbow. Let's try using silhouette analysis.

From https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py:

Silhouette analysis can be used to study the separation distance between the resulting clusters. The silhouette plot displays a measure of how close each point in one cluster is to points in the neighboring clusters and thus provides a way to assess parameters like number of clusters visually. This measure has a range of [-1, 1].

Silhouette coefficients (as these values are referred to as) near +1 indicate that the sample is far away from the neighboring clusters. A value of 0 indicates that the sample is on or very close to the decision boundary between two neighboring clusters and negative values indicate that those samples might have been assigned to the wrong cluster.



In [ ]:
# Code from https://towardsdatascience.com/k-means-clustering-algorithm-applications-evaluation-methods-and-drawbacks-aa03e644b48a
# Per the article, a good number of clusters will have a well above 0.5 silhouette 
# average score as well as all of the clusters have higher than the average score

for i, k in enumerate(range(2,11)):
    #fig, (ax1, ax2) = plt.subplots(1, 2)
    fig, ax1 = plt.subplots(1, 1)
    fig.set_size_inches(18, 7)
    
    # Run the Kmeans algorithm
    km = KMeans(n_clusters=k)
    labels = km.fit_predict(scaled_data)
    centroids = km.cluster_centers_

    # Get silhouette samples
    silhouette_vals = silhouette_samples(scaled_data, labels)

    # Silhouette plot
    y_ticks = []
    y_lower, y_upper = 0, 0
    for i, cluster in enumerate(np.unique(labels)):
        cluster_silhouette_vals = silhouette_vals[labels == cluster]
        cluster_silhouette_vals.sort()
        y_upper += len(cluster_silhouette_vals)
        ax1.barh(range(y_lower, y_upper), cluster_silhouette_vals, edgecolor='none', height=1)
        ax1.text(-0.03, (y_lower + y_upper) / 2, str(i + 1))
        y_lower += len(cluster_silhouette_vals)

    # Get the average silhouette score and plot it
    avg_score = np.mean(silhouette_vals)
    ax1.axvline(avg_score, linestyle='--', linewidth=2, color='green')
    ax1.set_yticks([])
    ax1.set_xlim([-0.1, 1])
    ax1.set_xlabel('Silhouette coefficient values')
    ax1.set_ylabel('Cluster labels')
    ax1.set_title('Silhouette plot for the various clusters', y=1.02);
    print('Average silhouette score for {} clusters is {}'.format(k,avg_score))
    
   # plt.tight_layout()
    plt.suptitle(f'Silhouette analysis using k = {k}',
                 fontsize=16, fontweight='semibold', y=1.05);


None of our clusters have a score higher than 0.5. Based on this, I see three possible conclusions:
1. K-Means is probably not the right algorithm for us
2. Maybe there is only a single cluster in the data, but I doubt this. If nothing else, counties with large cities should cluster together.
3. Maybe there are too many dimensions? Can try running PCA on the data, then K-Means

### Agglomerative Clustering
Agglomerative clustring performs a bottom up approach where each observation starts in its own cluster and clusters are successively merged together.

There are 4 types of linkages we will consider:
- Single: minimum distance between clusters
- Complete: maximum distance between clusters
- Average: average distance between clusters
- Ward: difference between:
- - The total within-cluster sum of squares for the two clusters seperately
- - and
- - The within-cluster sum of squares resulting from merging the two clusters

We are using Ward's Method to start because it tends to create equal sized clusters and is effective for noisy data. 

The output of the ward function is as follows:
- Column 1 and 2 are child nodes
- Column 3 is distance
- Column 4 is the number of leaf nodes merged

Every node 58 and higher are nodes that are created from merges. Node id 58 is created from the first line of the ward result, node id 59 is the second line, etc.

In [ ]:
plt.figure(figsize=(10,6))
cls1 = ward(scaled_data)
dendrogram(cls1)#,orientation='left')
plt.title('Dendrogram using Ward Linkage')
plt.show()

print(cls1)

Based on the dendrogram we are going to assign the data to 3 clusters, so now we will run the clusters to get the assignment value

In [ ]:
cls_w = AgglomerativeClustering(n_clusters=3, linkage='ward')
cls_assignment_w = cls_w.fit_predict(scaled_data)
cls_assignment_w

In [ ]:
#Zip together the mapper and the cluster assignment
county_cluster = list(zip(county_list_mapper, list(cls_assignment_w)))

cluster_df = pd.DataFrame(county_cluster, columns=['county_name', 'agglom_ward_cluster'])
cluster_df


In [ ]:
#Show the clusters on a map
ca_all = pd.merge(counties,cluster_df,how='inner',on='county_name')

In [ ]:
alt.Chart(ca_all).mark_geoshape().encode(
    tooltip='NAME',
    color='agglom_ward_cluster:N'
).properties(title='Agglomerative Clusters - Ward Linkage - 2018 Data')

Out of curiousity, let's see if we get similar clusterings if we use different agglomerative clustering linkage methods 

In [ ]:
#Complete linkage method
plt.figure(figsize=(10,6))
cls2 = complete(scaled_data)
dendrogram(cls2)#,orientation='left')
plt.title('Dendrogram using Complete Linkage')
plt.show()

#print(cls2)

With complete linkage it seems that 5 clusters are more appropriate

In [ ]:

cls_c = AgglomerativeClustering(n_clusters=5, linkage='complete')
cls_assignment_c = cls_c.fit_predict(scaled_data)

county_cluster = list(zip(county_list_mapper, list(cls_assignment_c)))

new_df = pd.DataFrame(county_cluster, columns=['county_name', 'agglom_complete_cluster'])

ca_all = ca_all.merge(new_df, how = 'inner', on = 'county_name')

In [ ]:
#Average Linkage Method
plt.figure(figsize=(10,6))
cls3 = average(scaled_data)
dendrogram(cls3)#,orientation='left')
plt.title('Dendrogram using Average Linkage')
plt.show()

#print(cls3)

With average linkage it seems that 5 clusters are appropriate

In [ ]:

cls_a = AgglomerativeClustering(n_clusters=5, linkage='average')
cls_assignment_a = cls_a.fit_predict(scaled_data)

county_cluster = list(zip(county_list_mapper, list(cls_assignment_a)))

new_df = pd.DataFrame(county_cluster, columns=['county_name', 'agglom_average_cluster'])

ca_all = ca_all.merge(new_df, how = 'inner', on = 'county_name')

In [ ]:
#Single linkage method
plt.figure(figsize=(10,6))
cls4 = single(scaled_data)
dendrogram(cls4)#,orientation='left')
plt.title('Dendrogram using Single Linkage')
plt.show()

#print(cls4)

Using single linkage 2 clusters seems most appropriate

In [ ]:
cls_s = AgglomerativeClustering(n_clusters=2, linkage='single')
cls_assignment_s = cls_s.fit_predict(scaled_data)

county_cluster = list(zip(county_list_mapper, list(cls_assignment_s)))

new_df = pd.DataFrame(county_cluster, columns=['county_name', 'agglom_single_cluster'])

ca_all = ca_all.merge(new_df, how = 'inner', on = 'county_name')

Show the rest of the agglomerative clustering maps

In [ ]:
#Show Complete Linkage Cluster Map

complete = alt.Chart(ca_all).mark_geoshape().encode(
    tooltip='NAME',
    color='agglom_complete_cluster:N'
).properties(title='Agglomerative Clusters - Complete Linkage - 2018 Data')

complete

In [ ]:
#Show average linkage cluster map
average = alt.Chart(ca_all).mark_geoshape().encode(
    tooltip='NAME',
    color='agglom_average_cluster:N'
).properties(title='Agglomerative Clusters - Average Linkage - 2018 Data')
average

In [ ]:
#Show single linkage cluster map
single = alt.Chart(ca_all).mark_geoshape().encode(
    tooltip='NAME',
    color='agglom_single_cluster:N'
).properties(title='Agglomerative Clusters - Single Linkage - 2018 Data')
single


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f6c76417-5fde-42f3-8920-755838dec3fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>